# FIT5196 Task 3 in Assessment 2
#### Student Name: Chuangfu Xie
#### Student ID: 27771539

Date: 10/05/2018

Version: 1.0

Environment: Python 3.6.4
Packages list:
1. Pandas 0.22.0  
2. numpy 1.14.0
3. sklearn 0.19.1

In [ ]:
import sys
print (sys.version)

## 1.  Import libraries 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt

## 2. Inspection

In [ ]:
df = pd.read_csv("./dataset3_with_missing.csv")

In [ ]:
# take a peek at the dataset
df.head()

First, let's check on the missing values in each columns by `check_mvs()`:

In [ ]:
def check_mvs(df):
    print('MVs within total ' + str(len(df)) + ' records:')
    for col in df.columns.tolist():
        total = df[col].isnull().sum()
        if total!=0:
            print(col + ': ' + str(total))
check_mvs(df)

## 3. Analysing and Imputation

First, let's look at the 3 related columns: `sqft_living`, `sqft_above`, `sqft_basement`.

In [ ]:
rel_col = ['sqft_living','sqft_above','sqft_basement']
df[rel_col].head()

It seems that values of these 3 columns are following a arithmetic formula:  
$$S_{living} = S_{above} + S_{basement} $$
Let's check:

In [ ]:
check_mvs(df[df.sqft_living == (df.sqft_above + df.sqft_basement)])

Look like records following this formul doesn't have any missing values in these 3 columns. How above the opposite?

In [ ]:
check_mvs(df[df.sqft_living != (df.sqft_above + df.sqft_basement)])

Obviously, any missing values contains in a record will not satify this formula.  
However, to satisfy this equation, there is a limitation that **only 1 missing value** is allowed spread within these 3 columns in one single record, otherwise, any value can imputed to satisfy this equation. Hence, we need to check the number of missing values in each record. To do this, we can make use of `df.dropna()` method by setting threshold as 2: 

In [ ]:
len(df[df.sqft_living != (df.sqft_above + df.sqft_basement)].dropna(thresh=2))

Or, we can simply assume that if all extracted records satisfy the above limitation, then we have the proof:
$$N_{living}(MVs) + N_{above}(MVs) + N_{basement}(MVs) = N_{relSqft}(MVs)$$
$$66 + 67 + 67 = 200$$
Now we can impute data following this equation:

In [ ]:
# sqft_living
df.sqft_living.fillna((df.sqft_above + df.sqft_basement), inplace=True)
# sqft_above
df.sqft_above.fillna((df.sqft_living - df.sqft_basement), inplace=True)
# sqft_basement
df.sqft_basement.fillna((df.sqft_living - df.sqft_above), inplace=True)
# Double-check
check_mvs(df)

Now, we can move on to deal with the problem in bathrooms.
To tackle this problem, as the number of bathrooms in a house is not simply relied on any formula, we can use **linear regression** to approximately predict the potential values.  

However, before employ linear regression, we need to find out the linear correlation based on **Pearson correlation coefficient** between different columns. Hence, we use `check_corr()` to check:

In [ ]:
def check_cor(df, target_col):
    results = dict()
    tmp = df.dropna()
    for col in tmp.columns.tolist():
        x = tmp[col]
        try:
            results[col + '_' + target_col] = pearsonr(x, tmp[target_col])[0]
        except:
            print("Can't calcalute due to: " + col + ' is ' + tmp[col].dtype.str)
    return results

In [ ]:
check_cor(df, 'bathrooms')

Now we can have the correlation coefficient with respect to the column correlated to `bathrooms`. Let's give a threshold to sift those correlation are moderate positive, that equals to **0.5** (according to **Pearson's rule of thumb**):

In [ ]:
cor_dict = check_cor(df, 'bathrooms')
for pair in cor_dict:
    if cor_dict[pair] > 0.5:
        print(pair)

There are 7 columns which have the correlation coefficient greater than the threshold. However, in terms of interpretation of the size of a correlation, some literature has given guideline on how to interprete the correlation coefficient. However, the interpretation actually depends on the context and purposes. We should **first use a scattor plot to establish if the data indicates linear relationship**.

Now, let's first plot the scatter plot by `plot_scatter`: 

In [ ]:
def plot_scatter(df, col, col_list):
    if len(col_list)%2!=0:
        nr=len(col_list)//2+1
    else:
        nr=len(col_list)//2
    fig, axes = plt.subplots(nrows=nr, ncols=2, figsize=(10, 10))
    r, c = 0, 0
    for i in col_list:
        if c>1:
            c=0
            r+=1
        if r<nr:
            axes[r, c].yaxis.set_major_locator(plt.NullLocator())
            axes[r, c].xaxis.set_major_formatter(plt.NullFormatter())
            axes[r, c].scatter(df[i], df[col], marker='p', s=2)
            axes[r, c].set_title(i + ' and ' + col)
            c+=1
    plt.show()

In [ ]:
# prepare for data: no missing value allowed
df_dropna = df.dropna()
cor_col = ['bathrooms', 'price', 'bedrooms', 'floors', 
           'grade', 'sqft_living', 'sqft_above', 'yr_built']
plot_scatter(df_dropna, 'bathrooms', cor_col)

From above, we can find these columns scatter plot are approximately having a linear relationship with bathrooms:
* <font color='blue'>**price**</font>
* <font color='blue'>**grade**</font>
* <font color='blue'>**sqft_living**</font>
* <font color='blue'>**sqft_above**</font>

However, with a closed examine these graphs, their relations have not been linearised. Hence, we need to employ **data transformation**:

Based on **Tukey's Ladder of Powers**, we employ **Power transformation** to improve the linearity between variables:
![Tukey's Ladder of Powers](http://www.math.smith.edu/~bbaumer/mth247/labs/tukey_bulge.png)
Follow rule shown above, we should transform data by **going down-ladder** for x, which is apply **log transformation**:

In [ ]:
col_trans = ['bathrooms', 'price', 'grade', 'sqft_living', 'sqft_above']
df_trans = df_dropna.copy() # make a copy
# log transformation
for i in col_trans:
    if i!='bathrooms':
        df_trans[i] = np.log(df_trans[i]) #log transformation

Now, let's check again the scatter plot:

In [ ]:
plot_scatter(df_trans, 'bathrooms', col_trans)

In [ ]:
before = check_cor(df[col_trans], 'bathrooms')
after = check_cor(df_trans[col_trans], 'bathrooms')
for i in before:
    if i!='bathrooms_bathrooms':
        print(i + ' improved by: ' + str(round(100*(after[i]/before[i]-1),2))+'%')

It seems that the linearity has been improved to some degree.

Before we do employ this **supervised learning** technique to predict data, we first need to train our linear model by using the data without records containing any `NaN`s. After that, we can use our train dataset to fit our model.  

In [ ]:
df_train = df_trans[col_trans]
col_list = df_train.columns.tolist()
x_train, x_test, y_train, y_test = train_test_split(df_train[col_list[1:]],
                                                    df_train['bathrooms'],
                                                    random_state = 42)
model = LinearRegression()
model.fit(x_train, y_train)

Then we can use $R^2$ score to see **how close the data are to the fitted regression line**, especially in this case.

In [ ]:
# verify by R^2 score
model.score(x_test, y_test)

The $R^2$ score seems to be fine, now we can use the full dataset `df` to predict potential values for <font color='blue'>**bathrooms**</font>:

In [ ]:
col_predict = ['price', 'grade', 'sqft_living', 'sqft_above']
X_test = df[col_predict].copy()
for i in col_predict:
    if i!='bathrooms':
        X_test[i] = np.log(X_test[i])
model.predict(X_test)

Look like the value with so many digits doesn't make sense to this column, but we can use `np.round()` to keep only 1 digit, making it similar to the other:

In [ ]:
predict_values = np.round(model.predict(X_test),1)

Now we can replace all the `NaN` values in <font color='blue'>**bathrooms**</font> with our predictive values:

In [ ]:
nan_index = df[df.bathrooms.isnull()].index.values
df.loc[nan_index, 'bathrooms']= predict_values[nan_index]
#Double check
check_mvs(df)

## 4. Export CSV 

In [ ]:
df.to_csv('./dataset3_solution.csv',index=False)

## 5. Summary

In this task, the noteworthy point is that the relationships between data are not always obvious. Some relationships are buried so deep that the scatter plot might be a good choice to dig out. However, some data distribution might so skewed that we also need transformation to linearise it into optimal form. With $R^2$ score, we can verify how well our model has been fit.

## Reference

* Thoma M.& Sacha. *Calculating Pearson correlation and significance in Python*. Retrieved from: [https://stackoverflow.com/a/3949282](https://stackoverflow.com/a/3949282)
* Rumsey J.D. Statistics For Dummies.2ed. *HOW TO INTERPRET A CORRELATION COEFFICIENT R
 Statistics For Dummies*[http://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/](http://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/)